<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_2/blob/main/DINOv2_olympia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Olympia_new_DINOv2 (simple classification)**


In [1]:
# prompt: gdrivewom

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Preprocess dataset**

image size 224px

Letterbox

In [3]:
# prompt: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zipを/contentに解凍

import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def letterbox(img, new_shape=(224, 224), color=(114, 114, 114)):
    # 元の形状
    shape = img.shape[:2]  # current shape [height, width]

    # スケール比を計算
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])

    # 新しいサイズを計算（アスペクト比を維持）
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))

    # パディングの計算
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]

    # パディングを均等に分配
    dw /= 2
    dh /= 2

    # リサイズ
    img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)

    # パディングを追加
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return img

def create_dataset_structure(grav_path, cont_path, output_path, test_size=0.2):
    # 出力ディレクトリの作成
    os.makedirs(output_path, exist_ok=True)
    train_dir = os.path.join(output_path, 'train')
    val_dir = os.path.join(output_path, 'val')

    for dir_path in [train_dir, val_dir]:
        os.makedirs(os.path.join(dir_path, 'grav'), exist_ok=True)
        os.makedirs(os.path.join(dir_path, 'cont'), exist_ok=True)

    # ファイルの処理
    grav_files = [f for f in os.listdir(grav_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    cont_files = [f for f in os.listdir(cont_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # データ分割
    grav_train, grav_val = train_test_split(grav_files, test_size=test_size, random_state=42)
    cont_train, cont_val = train_test_split(cont_files, test_size=test_size, random_state=42)

    def process_files(files, src_path, dest_class_path, desc):
        for f in tqdm(files, desc=desc):
            # 画像の読み込み
            img_path = os.path.join(src_path, f)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue

            # BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # レターボックス処理
            img = letterbox(img, new_shape=(224, 224))

            # RGB to BGR (OpenCVで保存するため)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # 保存
            out_path = os.path.join(dest_class_path, f)
            cv2.imwrite(out_path, img)

    # trainデータの処理
    print("Processing training data...")
    process_files(grav_train, grav_path, os.path.join(train_dir, 'grav'), "Processing grav train")
    process_files(cont_train, cont_path, os.path.join(train_dir, 'cont'), "Processing cont train")

    # valデータの処理
    print("\nProcessing validation data...")
    process_files(grav_val, grav_path, os.path.join(val_dir, 'grav'), "Processing grav val")
    process_files(cont_val, cont_path, os.path.join(val_dir, 'cont'), "Processing cont val")

    return len(grav_train), len(grav_val), len(cont_train), len(cont_val)

# パスの設定と実行
grav_path = '/content/Olympia_new/活動期写真'
cont_path = '/content/Olympia_new/コントロール写真'
output_path = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/dinov2_224px'

# 実行
train_grav, val_grav, train_cont, val_cont = create_dataset_structure(
    grav_path, cont_path, output_path
)

print(f"\nDataset creation completed!")
print(f"Train data: Grav={train_grav}, Cont={train_cont}")
print(f"Val data: Grav={val_grav}, Cont={val_cont}")

Processing training data...


Processing cont train: 100%|██████████| 827/827 [01:03<00:00, 12.97it/s]



Processing validation data...


Processing cont val: 100%|██████████| 207/207 [00:18<00:00, 11.18it/s]


Dataset creation completed!
Train data: Grav=829, Cont=827
Val data: Grav=208, Cont=207


In [3]:
!git clone https://github.com/facebookresearch/dinov2.git
%cd dinov2
!pip install -q submitit
!pip install -q xformers==0.0.18
!pip install -q -r requirements.txt

# PYTHONPATHの設定
import os
os.environ['PYTHONPATH'] = '.'

Cloning into 'dinov2'...
remote: Enumerating objects: 485, done.
remote: Total 485 (delta 0), reused 0 (delta 0), pack-reused 485 (from 1)
Receiving objects: 100% (485/485), 1.11 MiB | 3.38 MiB/s, done.
Resolving deltas: 100% (237/237), done.
/content/dinov2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

##**Create metadata**

In [10]:
#メタデータ生成
import numpy as np

class ThyroidEyeDataset:
    def __init__(self, root_dir, split='train'):
        self.root_dir = os.path.join(root_dir, split)
        self.split = split
        self.classes = ['cont', 'grav']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            if os.path.isdir(class_dir):
                for fname in os.listdir(class_dir):
                    if fname.lower().endswith(('.png', '.jpg', '.jpeg', 'JPG')):
                        path = os.path.join(class_dir, fname)
                        self.samples.append((path, self.class_to_idx[class_name]))

    def dump_extra(self, extra_dir):
        os.makedirs(extra_dir, exist_ok=True)

        class_ids = np.array([idx for idx in range(len(self.classes))])
        class_names = np.array(self.classes)
        entries = np.array([path for path, _ in self.samples])

        np.save(os.path.join(extra_dir, f'class-ids-{self.split.upper()}.npy'), class_ids)
        np.save(os.path.join(extra_dir, f'class-names-{self.split.upper()}.npy'), class_names)
        np.save(os.path.join(extra_dir, f'entries-{self.split.upper()}.npy'), entries)


# データセットとメタデータのパスを設定
dataset_path = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/dinov2_224px'
metadata_path = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/dinov2_224px/metadata'

# メタデータを生成
for split in ['train', 'val']:
    dataset = ThyroidEyeDataset(dataset_path, split=split)
    dataset.dump_extra(metadata_path)

##**Finetune DINOv2**

In [1]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

# 既存のdinov2ディレクトリを削除（もし存在する場合）
!rm -rf dinov2

# DINOv2リポジトリのクローン
!git clone https://github.com/facebookresearch/dinov2.git

# dinov2ディレクトリに移動
%cd dinov2

# 必要なパッケージのインストール
!pip install -q submitit
!pip install -q xformers==0.0.18
!pip install -q -r requirements.txt
!pip install -e .

Mounted at /content/drive
Cloning into 'dinov2'...
remote: Enumerating objects: 485, done.
remote: Total 485 (delta 0), reused 0 (delta 0), pack-reused 485 (from 1)
Receiving objects: 100% (485/485), 1.11 MiB | 20.24 MiB/s, done.
Resolving deltas: 100% (237/237), done.
/content/dinov2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00

In [ ]:
!PYTHONPATH=$PYTHONPATH:/content/dinov2 python dinov2/run/eval/linear.py \
    --config-file dinov2/configs/eval/vits14_pretrain.yaml \
    --pretrained-weights https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth \
    --train-dataset ImageNet:split=TRAIN:root={dataset_path}:extra={metadata_path} \
    --val-dataset ImageNet:split=VAL:root={dataset_path}:extra={metadata_path} \
    --output-dir /content/drive/MyDrive/Deep_learning/Olympia_dataset/dinov2_output

2024-12-16 09:14:11.488624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 09:14:11.507558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 09:14:11.513491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 09:14:11.527842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 09:14:13.089794: W tensorflow/compiler/tf2